In [ ]:
from json import *

with open('task1.txt','r',encoding='utf-8',errors='ignore')as f:
    all_entry=[JSONDecoder().decode(x) for x in f.readlines()]

all_idtext={}
for i in all_entry:
    all_idtext[i['id']]=i['text']

with open('smm4h-EMNLP-task1-trainingset.csv','r',encoding='utf-8',errors='ignore')as f:
    all_labeled=[x.replace('\n','').split('\t') for x in f.readlines()]


with open('smm4h-EMNLP-task1-testset.tsv','r',encoding='utf-8',errors='ignore')as f:
    all_idtext_test=[x.replace('\n','').split('\t')[2] for x in f.readlines()]
with open('smm4h-EMNLP-task1-testset.tsv','r',encoding='utf-8',errors='ignore')as f:
    all_id_test=[x.replace('\n','').split('\t')[0] for x in f.readlines()]

all_user={}
for i in all_labeled:
    all_user[int(i[0])]=int(i[2])

In [ ]:
len(all_id_test)

In [ ]:

with open('meddra_all_se.tsv','r',encoding='utf-8',errors='ignore')as f:
    adrlexicon={tuple(x.replace('\n','').split('\t')[5].lower().split()):1 for x in f.readlines()}

In [ ]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [ ]:
from nltk.corpus import sentiwordnet as swn


In [ ]:
import numpy as np

In [ ]:
all_text=[]
all_text_lower=[]
all_senti=[]
all_label=[]
for i in all_idtext:
    if i in all_user:
        tscore=[]
        tp=tknzr.tokenize(all_idtext[i])
        for m in range(len(tp)):
            if tp[m][0]=='@':
                tp[m]='@USER'
            if 'http' in tp[m]:
                tp[m]='URL'
            nk=list(swn.senti_synsets(tp[m]))
            if len(nk)>0:
                tscore.append(np.mean([x.pos_score()-x.neg_score() for x in nk]))
            else:
                tscore.append(0)
        all_senti.append(tscore) 
        all_text_lower.append([x.lower() for x in tp])
        all_text.append(tp)
        all_label.append(all_user[i])

In [ ]:
all_adr=[]
for i in all_text_lower:
    adr_feat=[0]*len(i)
    for termlen in range(5):
        for index in range(len(i)-termlen+1):
            if tuple(i[index:index+termlen]) in adrlexicon:
                for p in range(termlen):
                    adr_feat[index+p]=1
    all_adr.append(adr_feat)


In [ ]:
all_text_test=[]
all_text_lower_test=[]
all_senti_test=[]
for i in all_idtext_test:
    tscore=[]
    tp=tknzr.tokenize(i)
    for m in range(len(tp)):
        if tp[m][0]=='@':
            tp[m]='@USER'
        if 'http' in tp[m]:
            tp[m]='URL'
        nk=list(swn.senti_synsets(tp[m]))
        if len(nk)>0:
            tscore.append(np.mean([x.pos_score()-x.neg_score() for x in nk]))
        else:
            tscore.append(0)
    all_senti_test.append(tscore) 
    all_text_lower_test.append([x.lower() for x in tp])
    all_text_test.append(tp)


In [ ]:
all_adr_test=[]
for i in all_text_lower_test:
    adr_feat=[0]*len(i)
    for termlen in range(5):
        for index in range(len(i)-termlen+1):
            if tuple(i[index:index+termlen]) in adrlexicon:
                for p in range(termlen):
                    adr_feat[index+p]=1
    all_adr_test.append(adr_feat)
    

In [ ]:
import emoji
from nltk import pos_tag

smiley=[':-(',':-)',':(',':)',':\')']
def edit_pos(pos):
    
    for i in range(len(pos)):
        pos[i]=list(pos[i])

        if pos[i][0]=='@USERNAME':
            pos[i][1]='@'
        if pos[i][0]=='TRIGGERWORD':
            pos[i][1]='TRIGGERWORD'
        if pos[i][0] in smiley:
            pos[i][1]='E'
        if pos[i][0][0]=='#':
            pos[i][1]='#'
            
        if pos[i][0][0] in emoji.UNICODE_EMOJI:
            pos[i][1]='E'
    return pos    

all_pos=[]
for i in range(len(all_text)):
    all_pos.append(edit_pos(pos_tag(all_text[i])))  
all_pos_test=[]
for i in range(len(all_text_test)):
    all_pos_test.append(edit_pos(pos_tag(all_text_test[i])))  


In [ ]:
pos_dict={'PADDING':0}
for i in all_pos:
    for j in i:
        if j[1] not in pos_dict:
            pos_dict[j[1]]=len(pos_dict)
for i in all_pos_test:
    for j in i:
        if j[1] not in pos_dict:
            pos_dict[j[1]]=len(pos_dict)
            
word_dict0={'PADDING':[0, 999999]}

char_dict={'PADDING':0}


for i in all_pos:
    for j in i:
        if j[0] not in word_dict0:
            word_dict0[j[0]]=[len(word_dict0),1]
        else:
            word_dict0[j[0]][1]+=1
        for char in j[0]:
            if char not in char_dict:
                char_dict[char]=len(char_dict)
                
                
for i in all_pos_test:
    for j in i:
        if j[0] not in word_dict0:
            word_dict0[j[0]]=[len(word_dict0),1]
        else:
            word_dict0[j[0]][1]+=1
        for char in j[0]:
            if char not in char_dict:
                char_dict[char]=len(char_dict)


In [ ]:
word_dict={'PADDING':[0, 999999]}
for i in word_dict0:
    if word_dict0[i][1]>=1:
        word_dict[i]=[len(word_dict),word_dict0[i][1]]
print(len(word_dict))

In [ ]:
len(char_dict)

In [ ]:
full_data=[]
full_pos=[]
full_adr=[]
full_senti=[]
full_char=[]

for i in all_pos:
    tp=[]

    postag=[]
    chartp=[]

    for j in i:
        if j[0] in word_dict:
            tp.append(word_dict[j[0]][0])
            postag.append(pos_dict[j[1]])
            chartp0=[char_dict[char] for char in j[0]][:15]
            chartp.append(chartp0+[0]*(15-len(chartp0)))
         
    tp=tp[:80]
    postag=postag[:80]
    chartp=chartp[:80]
    
    full_char.append(chartp+[[0]*15]*(80-len(chartp)))
    full_pos.append(postag+[0]*(80-len(postag)))
    full_data.append(tp+[0]*(80-len(tp)))

for i in all_senti:
    tp=i[:80]
    full_senti.append(tp+[0]*(80-len(tp)))
    
for i in all_adr:
    tp=i[:80]
    full_adr.append(tp+[0]*(80-len(tp)))

In [ ]:
test_data=[]
test_pos=[]
test_adr=[]
test_senti=[]
test_char=[]

for i in all_pos_test:
    tp=[]

    postag=[]
    chartp=[]

    for j in i:
        if j[0] in word_dict:
            tp.append(word_dict[j[0]][0])
            postag.append(pos_dict[j[1]])
            chartp0=[char_dict[char] for char in j[0]][:15]
            chartp.append(chartp0+[0]*(15-len(chartp0)))
         
    tp=tp[:80]
    postag=postag[:80]
    chartp=chartp[:80]
    
    test_char.append(chartp+[[0]*15]*(80-len(chartp)))
    test_pos.append(postag+[0]*(80-len(postag)))
    test_data.append(tp+[0]*(80-len(tp)))

for i in all_senti_test:
    tp=i[:80]
    test_senti.append(tp+[0]*(80-len(tp)))
    
for i in all_adr_test:
    tp=i[:80]
    test_adr.append(tp+[0]*(80-len(tp)))

In [ ]:
len(all_label)

In [ ]:
import numpy as np
index=list(range(len(all_label)))
import random
random.shuffle(index)

train_data=np.array(full_data,dtype='float32')
test_data=np.array(test_data,dtype='float32')

train_senti=np.array(full_senti,dtype='float32')
test_senti=np.array(test_senti,dtype='float32')

train_adr=np.array(full_adr,dtype='float32')
test_adr=np.array(test_adr,dtype='float32')

train_pos=np.array(full_pos,dtype='float32')
test_pos=np.array(test_pos,dtype='float32')


train_char=np.array(full_char,dtype='float32')
test_char=np.array(test_char,dtype='float32')


train_label=np.array(all_label,dtype='float32')
#test_label=np.array(all_label,dtype='float32')[index[7500:]]

In [ ]:
test_char.shape

In [ ]:
import numpy as np
embdict=dict()
plo=0
with open('/home/wuch/word2vec_twitter_model.bin','rb')as f:
    header = f.readline()
    vocab_size, layer1_size = map(int, header.split())
    binary_len = np.dtype('float32').itemsize * layer1_size
    for line in range(vocab_size):
        word = []
        while True:
            ch = f.read(1).decode('utf-8','ignore')
            if ch ==' ':
                word = ''.join(word)
                break
            if ch != '\n':
                word.append(ch)
        if len(word) != 0:
            tp= np.fromstring(f.read(binary_len), dtype='float32')
            if word in word_dict:
                embdict[word]=tp.tolist()
                if plo %1000==0:
                    print(plo,line,word)
                plo+=1
                #print(word,tp)
        else:
            f.read(binary_len)


In [ ]:
from numpy.linalg import cholesky
print(len(embdict),len(word_dict))
print(len(word_dict))
lister=[0]*len(word_dict)
xp=np.zeros(400,dtype='float32')

cand=[]
for i in embdict.keys():
    lister[word_dict[i][0]]=np.array(embdict[i],dtype='float32')
    cand.append(lister[word_dict[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(lister)):
    if type(lister[i])==int:
        lister[i]=np.reshape(norm, 400)
lister[0]=np.zeros(400,dtype='float32')
lister=np.array(lister,dtype='float32')
print(lister.shape)



In [ ]:

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, concatenate
from keras.layers import Dense, Input, Flatten, average,Lambda
from keras.layers import Convolution1D, MaxPooling1D,Conv1D,GlobalMaxPooling1D, AveragePooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from keras.utils import plot_model
import numpy as np
from sklearn.metrics import accuracy_score, classification_report,log_loss,mean_squared_error,mean_absolute_error



In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [ ]:
from keras.layers import *
from keras import initializers

from keras.optimizers import Optimizer


In [ ]:
from keras import backend as K
from keras.engine.topology import Layer

class Attention(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
        
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
                
    def call(self, x):
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))    
        A = K.softmax(A)
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [ ]:
MAX_SENT_LENGTH=80
MAX_CHAR=15
import random
all_pred2=[]
droprate=0.2

for times in range(20):
    embedding_layer = Embedding(len(word_dict), 400,input_length=MAX_SENT_LENGTH,weights=[lister], trainable=True)
    
    
    review_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    review_embedded_sequences = embedding_layer(review_input)
    d_review_emb=Dropout(droprate)(review_embedded_sequences)
    
      
    embedding_layer_char = Embedding(len(char_dict), 100,input_length=MAX_CHAR,trainable=True)
    
    
    char_input = Input(shape=(MAX_CHAR,), dtype='int32')
    char_embedded_sequences = embedding_layer(char_input)
    d_char_emb=Dropout(droprate)(char_embedded_sequences)
    
    char_c_cnn = Convolution1D(nb_filter=100, filter_length=5, padding='same', activation='relu', strides=1)(d_char_emb)
    char_mp=GlobalMaxPooling1D()(char_c_cnn)
    
    charEncoder = Model(char_input, char_mp)
    
    char_all_input = Input(shape=(MAX_SENT_LENGTH, MAX_CHAR), dtype='int32')
    char_encoder = TimeDistributed(charEncoder)(char_all_input)
    
    
    pos_embedding_layer = Embedding(len(pos_dict), 100,input_length=MAX_SENT_LENGTH, trainable=True)
    
    
    pos_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    

    
    pos_embedded_sequences = pos_embedding_layer(pos_input)
    d_pos_emb=Dropout(droprate)(pos_embedded_sequences)
    
    
    adr_input = Input(shape=(MAX_SENT_LENGTH,1), dtype='float32')
    adr_dense=Dense(25,activation='relu')(adr_input)
    
    senti_input = Input(shape=(MAX_SENT_LENGTH,1), dtype='float32')
    senti_dense=Dense(25,activation='relu')(senti_input)
    
    
    
    ice_concat=concatenate([d_review_emb,d_pos_emb,char_encoder,adr_dense,senti_dense])
      
    
    review_w_lstm = Bidirectional(LSTM(200, return_sequences=True))(ice_concat)
    
    review_d_w_lstm = Dropout(droprate)(review_w_lstm)
    
    review_d_w_lstm1 = Attention(16,16)([review_d_w_lstm,review_d_w_lstm,review_d_w_lstm])
    
    review_w_cnn = Convolution1D(nb_filter=400, filter_length=3, padding='same', activation='relu', strides=1)(review_d_w_lstm1)
    
    review_d_w_cnn = Dropout(droprate)(review_w_cnn)
    

    
    attention = GlobalMaxPooling1D()(review_d_w_cnn )
   
    all_dense = Dense(200, activation='relu')(attention)   
    
    classifier = Dense(2, activation='softmax')(all_dense)
    
    
    model = Model([review_input,pos_input,char_all_input,adr_input,senti_input], [classifier])
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    

    for ep in range(3):
        model.fit([train_data,train_pos,train_char,np.expand_dims(train_adr,axis=2),np.expand_dims(train_senti,axis=2)],[to_categorical(train_label)],shuffle=True,class_weight='auto', batch_size=64, epochs=1,verbose=1)
        
        y_pred = model.predict([test_data, test_pos, test_char,np.expand_dims(test_adr,axis=2),np.expand_dims(test_senti,axis=2)], batch_size=64, verbose=1)
        all_pred2.append(y_pred)
        #y_pred = np.argmax(y_pred, axis=1)
        #y_true = test_label
        #cr = classification_report(y_true, y_pred,digits=4)
        #print(cr)
        

In [ ]:
all_pred3=[]
for i in range(len(all_pred2)):
    if i%3==2:
        all_pred3.append(all_pred2[i])

In [ ]:
len(all_pred3)

In [ ]:
test_pred=np.argmax(np.mean(all_pred3,axis=0),axis=1)

In [ ]:
with open('task1_pred.txt','w',encoding='utf-8',errors='ignore')as f:
    for i in range(len(test_pred)):
        f.write(str(all_id_test[i])+'\t'+str(test_pred[i])+'\n')

In [ ]:
with open('task1_pred.txt','w',encoding='utf-8',errors='ignore')as f:
    for i in range(len(test_pred)):
        f.write(str(all_idtext_test[i])+'\t'+str(test_pred[i])+'\n')